In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path
import numpy as np
import pandas as pd

#fastai
from fastai.imports import *
from fastai.torch_imports import *
from fastai.core import *
from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import * 
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

#pytorch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable as V


In [2]:
spacy.load('en')

In [3]:
# set theme 
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('monokai')

In [4]:
PATH=Path('data/hemingway')
PATH.mkdir(exist_ok=True)
Path(PATH/'train').mkdir(exist_ok=True)
Path(PATH/'val').mkdir(exist_ok=True)
Path(PATH/'test').mkdir(exist_ok=True)

In [5]:
str(PATH)

'data/hemingway'

## save full hemingway with code

In [6]:
file = open(Path('/notebooks/volume/full_hemingway.txt'), encoding='utf-8')
fulltext = file.read()

In [62]:
r = np.random.randint(1,len(fulltext))
fulltext[r:(r+300)]

'ic.”\n\nHe took a big gulp of coffee.\n\n“Aw, hell!” I said. “It’s too early in the morning.”\n\n“There you go. And you claim you want to be a writer, too. You’re only a\nnewspaper man. An expatriated newspaper man. You ought to be ironical\nthe minute you get out of bed. You ought to wake up with your mout'

In [8]:
LanguageModelData?

### split hemingway by simply by line

In [9]:
lng = len(fulltext)
trn,val,test = [fulltext[0:int(lng*i)] for i in [0.7,0.2,0.1]]

In [10]:
# save to files
def save_txt_file(name, txt):    
    f=open(PATH/name, mode='w', encoding='utf-8')
    f.write(txt)
    f.close()

In [11]:
save_txt_file('hemingway.trn.txt', trn)
save_txt_file('hemingway.val.txt', val)
save_txt_file('hemingway.test.txt', test)

# Data loader 

In [34]:
bs, bptt = 80, 70
TEXT = data.Field(lower=True, tokenize='spacy')
FILES = dict(train='hemingway.trn.txt', val='hemingway.val.txt', test='hemingway.text.txt')
md = LanguageModelData.from_text_files(PATH, TEXT, 'hemingway.trn.txt', 'hemingway.val.txt', 'hemingway.test.txt', bs=bs, bptt=bptt, min_freq=10)

In [13]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [14]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [79]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
                           dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = seq2seq_reg
learner.clip=0.3

In [80]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

 46%|████▌     | 27/59 [00:04<00:05,  5.86it/s, loss=5.8] 

Exception in thread Thread-54:
Traceback (most recent call last):
  File "/opt/conda/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/fastai/lib/python3.6/site-packages/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/opt/conda/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



epoch      trn_loss   val_loss                            
    0      5.51389    5.356874  
    1      5.4123     5.365284                            
    2      5.379502   5.350643                            
    3      5.378961   5.370271                            
    4      5.37285    5.357415                            
    5      5.365857   5.353315                            
    6      5.352339   5.33947                             
    7      5.224696   4.836269                            
    8      4.751812   4.243293                            
    9      4.356316   3.974646                            
    10     4.102002   3.843693                            
    11     3.955219   3.763725                            
    12     3.858894   3.701036                            
    13     3.810454   3.684139                            
    14     3.783921   3.682809                            



[3.682809]

In [17]:
learner.fit(3e-3, 2, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss                            
    0      3.812834   3.665374  
    1      3.776463   3.605549                            
    2      3.706881   3.559662                            



[3.5596619]

In [19]:
learner.fit(3e-3, 2, wds=1e-6, cycle_len=2, cycle_mult=2)

epoch      trn_loss   val_loss                            
    0      3.710703   3.531252  
    1      3.644434   3.490881                            
    2      3.644039   3.47948                             
    3      3.593937   3.409544                            
    4      3.532881   3.354022                            
    5      3.481076   3.336434                            



[3.3364341]

In [20]:
learner.fit(1e-3, 3, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss                            
    0      3.452714   3.334925  
    1      3.449813   3.312935                            
    2      3.438204   3.29424                             
    3      3.442936   3.28781                             
    4      3.415574   3.261163                            
    5      3.39094    3.230272                            
    6      3.367583   3.227625                            



[3.227625]

In [22]:
learner.fit(1e-3, 10, wds=1e-6, cycle_len=2, cycle_mult=2)

epoch      trn_loss   val_loss                            
    0      3.220329   3.005607  
    1      3.167204   2.972532                            
    2      3.174891   2.972322                            
    3      3.139277   2.936433                            
    4      3.109345   2.908558                            
    5      3.090588   2.90291                             
    6      3.120181   2.914815                            
    7      3.093592   2.884377                            
    8      3.084477   2.858088                            
    9      3.050072   2.827158                            
    10     3.013362   2.802395                            
    11     2.988102   2.790097                            
    12     2.977916   2.777191                            
    13     2.977269   2.771533                            
    14     3.006637   2.799065                            
    15     2.995758   2.775349                            
    16     2.971883   2

   138     1.899252   1.369292                            
   139     1.897105   1.364256                            
   140     1.841137   1.340441                            
   141     1.84326    1.329097                            
   142     1.868416   1.343942                            
   143     1.840042   1.312965                            
   144     1.874324   1.319287                            
   145     1.811396   1.282052                            
   146     1.762577   1.279383                            
   147     1.888861   1.319795                            
   148     1.845993   1.251248                            
   149     1.815043   1.234348                            
   150     1.762038   1.223986                            
   151     1.786494   1.218828                            
   152     1.720157   1.192243                            
   153     1.787934   1.220648                            
   154     1.75183    1.180705                          

KeyboardInterrupt: 

In [35]:
learner.save_encoder('hem_enc')

In [36]:
learner.load_encoder('hem_enc')

In [37]:
spacy_tok = spacy.load('en')
# TEXT = data.Field(lower=True, tokenize='spacy')

In [55]:
m=learner.model
ss="""supper I would go and see Catherine Barkley.\nI wished she were here now. I wished I were in Milan with her  """
s = [spacy_tok(ss)]
t=TEXT.numericalize(s)
# ' '.join(s[0])

In [81]:
def brave_test(ss, words_out):
    m=learner.model
    s = [spacy_tok(ss)]
    t=TEXT.numericalize(s)
    
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    m[0].bs=bs
    
    nexts = torch.topk(res[-1], 10)[1]
    [TEXT.vocab.itos[o] for o in to_np(nexts)]
    
    print(ss,"\n")
    for i in range(words_out):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        print(TEXT.vocab.itos[n.data[0]], end=' ')
        res,*_ = m(n[0].unsqueeze(0))
    print('...')

In [92]:
brave_test(""".  Model how are you. """, 15)

.  Model how are you.  

. ” “ i do n’t know . ” “ i do n’t know . ...


In [93]:
!find {PATH} -name '*.txt' | xargs cat | wc -w

find: '{TRN}': No such file or directory
0
